# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [15]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [16]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("Mapbox_Key")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [22]:
# Import the necessary CSVs to Pandas DataFrames
# YOUR CODE HERE!
sfo_path = Path('/Users/whitneyturner/Pyviz-Homework/Starter_Code/Data/sfo_neighborhoods_census_data.csv')
sfo_data = pd.read_csv(sfo_path, index_col="year")
file_path2 = Path('Data/neighborhoods_coordinates.csv')
location_df = pd.read_csv(file_path2)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [45]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    # YOUR CODE HERE!
    housing_units_per_year = sfo_data['housing_units'].groupby('year').mean()
    max1 = housing_units_per_year.max()
    min1 = housing_units_per_year.min()
    std1 = housing_units_per_year.std()
    housing_plot = housing_units_per_year.plot.bar(title = 'Housing Units in San Francisco per Year', y = std1, ylim = (min1 - 3000, max1 + 500))
    return housing_plot

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    # YOUR CODE HERE!
    avg_gross_rent_plot = avg_rent.plot.line(title = 'Average Gross Rent by Year', color = 'orange')
    avg_rent = pd.DataFrame(sfo_data['gross_rent'].groupby('year').mean())
    return avg_gross_rent_plot


def average_sales_price():
    """Average Sales Price Per Year."""
    
    # YOUR CODE HERE
    avg_per_foot = pd.DataFrame(sfo_data['sale_price_sqr_foot'].groupby('year').mean())
    avg_sales_price_plot = avg_per_foot.plot.line(title = 'Average Price per SqFt by Year')
    return avg_sales_price_plot


def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    # YOUR CODE HERE!
    df_costs = sfo_data.groupby([sfo_data.index, "neighborhood"]).mean()
    df_costs.reset_index(inplace=True)
    avg_price_by_neighborhhod_plot = df_costs.hvplot.line(x= "year", y = 'sale_price_sqr_foot', groupby = 'neighborhood')
    return avg_price_by_neighborhhod_plot



def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    # YOUR CODE HERE!
    df_exclusive = sfo_data.groupby("neighborhood").mean()
    df_exclusive1 = df_exclusive.sort_values('sale_price_sqr_foot', ascending = False)
    df_exclusive2 = df_exclusive1.reset_index()
    df_top_ten = df_exclusive2.head(10)
    most_expensive__neighborhoods_plot = df_top_ten.hvplot.bar(title = 'Top 10 Expensive Neighborhoods in San Francisco', x = 'neighborhood', y = 'sale_price_sqr_foot', ylabel = 'avg_price_per_sqr_foot',rot=90)
    return most_expensive_neighborhoods_plot

def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    # YOUR CODE HERE!
    df_expensive_neighborhoods_per_year = df_costs[df_costs["neighborhood"].isin(df_exclusive2["neighborhood"])]
    df_expensive_neighborhoods_per_year.head()
    most_expensive_neighborhoods_rent_sales_plot = df_costs.hvplot(
    y=["sale_price_sqr_foot", "gross_rent"],
    groupby = 'neighborhood',
    kind = "bar",
    ylabel="Cost of Dwelling",
    title="Home Purchase vs. Rental costs",
    x= 'year',
    rot=90
)
    return most_expensive_neighborhoods_rent_sales_plot
    
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""

    # YOUR CODE HERE!
    parallel_coordinates_plt = px.parallel_coordinates(df_expensive_neighborhoods_per_year, color = 'sale_price_sqr_foot')
    return parallel_coordinates_plt


def parallel_categories():
    """Parallel Categories Plot."""
    
    # YOUR CODE HERE!
    parallel_categories_plt = px.parallel_categories(
    df_top_ten,
    dimensions=['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent'],
    )
    return parallel_categories_plt


def neighborhood_map():
    """Neighborhood Map."""

    # YOUR CODE HERE!
    avg_neighborhood = sfo_data.groupby('neighborhood').mean()
    avg_neighborhood1 = avg_neighborhood.reset_index().drop(columns = 'neighborhood')
    avg_val_by_location = pd.concat([location_df, avg_neighborhood1], axis="columns", join = 'inner')

    neighborhood_map= px.scatter_mapbox(
    avg_val_by_location,
    lat="Lat",
    lon="Lon",
    color="Neighborhood"
    )
    return neighborhood_map

def sunburst():
    """Sunburst Plot."""
    
    # YOUR CODE HERE!
    fig = px.sunburst(
    df_expensive_neighborhoods_per_year,
    labels = 'neighborhoods',
    parents = 'year',
    values ='gross_rent'
    )
    return fig

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
# Create a Title for the Dashboard
# YOUR CODE HERE!
city_living_dashbord = '# How Expensive is it to live in San Francisco? Really Expensive!'

# Create a tab layout for the dashboard
# YOUR CODE HERE!
city_living_dashboard = pn.Tabs(
    ("Housing Units Per Year.", housing_plot), ("Average Gross Rent in San Francisco Per Year", avg_gross_rent_plot), ("Average Sales Price Per Year", avg_sales_price_plot),("Average Prices by Neighborhood", avg_price_by_neighborhhod_plot), ("")  
)

# Create the dashboard
# YOUR CODE HERE!

## Serve the Panel Dashboard

In [ ]:
# Serve the# dashboard
# YOUR CODE HERE!

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()

In [ ]:
# average_gross_rent()

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()